# Visualizations

In [1]:
# imports

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import pi

In [2]:
df = pd.read_csv('/Users/flanuer/Downloads/Lambda/Course_material/misc_datasets/non_dup_200k_song_aud_feat.csv')
drop_col = ['Unnamed: 0']
df = df.drop(drop_col, axis=1)

In [3]:
categories = df[:1]
N = len(categories)
angles = [n/float(N)*2*pi for n in range(N)]
angles += angles[:1]

In [11]:
df.id

0         0v1x6rN6JHRapa03JElljE
1         4Oun2ylbjFKMPTiaSbbCih
2         6UelLqGlWMcVH1E5c4H7lY
3         7ytR5pFWmSjzHJIeQkgog4
4         2SAqBLGA283SUiwJ3xOUVI
                   ...          
210119    7224Tp60srKUp78CkPfAo9
210120    0cvfSKcm9VeduwyYPrxtLx
210121    4TWlUuFk81NGUNKwndyS5Q
210122    5iGBXzOoRo4sBTy8wdzMyK
210123    7LNtyuekYHiZ99UxkrfCQR
Name: id, Length: 210124, dtype: object

# User Statistics
- Obtain user library
    - Track Audio Features
    - Artists
    - Playlists
    - Track Names
    - Genres
        - Can be gathered via artist
    - All time top songs/artist

In [5]:
from os import getenv
import spotipy
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyOAuth

In [6]:
load_dotenv()
client_secret = getenv('SPOTIFY_CLIENT_SECRET')
client_id = getenv('SPOTIFY_CLIENT_ID')
user_id = 'Agustinvargas'

# change for deplpoyment
uri = getenv('uri')

# Scopes: User top track; creates playlist
scope = getenv('SCOPE')
cache_path = '../../token_cache/'

spot_cc = spotipy.oauth2.SpotifyOAuth(
                                username=user_id,
                                client_id=client_id,
                                client_secret=client_secret,
                                scope=scope,
                                redirect_uri=uri,
                                show_dialog=True
                                )


spot_session = spotipy.Spotify(oauth_manager=spot_cc)

In [3]:
spot_cc.get_access_token()

<ipython-input-3-ae25c25473b4>:1: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  spot_cc.get_access_token()
Couldn't read cache at: .cache-Agustinvargas


Enter the URL you were redirected to:  https://pspotify.netlify.app/?code=AQCxsAclQxgnu2cEPtPAsztEVxxt-3wDyfsE5OQm5wQ-VSjibPnp3_km3DIBJxNr_ycvrKYorUSHTtgBOsVqDI9JWY7nSlZ_uqbNNQrCwatFj5Eya7WqUb4pQrG8zs1T2ut_hf8oJUKfIJFwmIsMg_t3jPea1AZzOGszri0umCNJu8mgh4f2IdsmW2zeYhMKOc5awNmKED0ZneI_o40JjYzah1GCxehEvVQl2u9ykZK-DqbX1IXgBZhikateW9U5WkFIuZPx0HEgxy6WMZ5kYKKALQvk3xzGODkGkkTdi1dv5B1voyIpPHljJ1uYEFn34jV4jJx6sWtQ1eZoslwPFfWXDBwdlLFdzuHzPG1iaRciSPR9x-LnV2LSaoNcR8sEv_30_9btfxHpeHElwE-r_HeHbgYIkZrcTay1Tnvijr0u0Hlu3CRAlEHYYe9fo3MIB8zp5hRzgDoXxPLM5WjLPx68-iqvy9OGwfLCiHULJ-Gu3wxBX462cV30Z9YkIyqo-qPIiLJWervkydvLZF9mbg-zbfJjMNqujAb0UDtJuhtjvGz9N13KL-t-gR7ELvtxCCZ6PryCG9kd8fvZ


{'access_token': 'BQBjO4IJYI2QtbAyQwm4kCPRIRPJYQs8x8Wtz2JBLhEZygqvd5RylwXlsdDQmQczv0nvpkSDvoH9TDLOOxLymfOYdLB8r4cqdv9oStI1HVRSHVRj1Dv-tRVu4fIvpkPYlTDIFqdyzF2dLFV_zHJ0PrqP60EfcB7ldNbfDLYjafvZ1iLKPGUJESo4_CIDF4jN6P8LCU4dYV_AEo7KVZrc_8vsOfAf4FHNK05ItK_9daMIwn2SN1ZKANl4wEso9bPyZE8BMo4tCNpPqbz4cbbE',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQD1o7VQjVb36XsZIodi6iK0keyhe5tJOqekGwy4vemqU2LUz0fMnewU420qMPnk0kIH9s0xxbPuuB0q7ji6kh5YCxXOiQtfzjw4ql1j26cixHnQS1_PzpuQJY66vXN8cTY',
 'scope': 'app-remote-control playlist-modify-private playlist-modify-public playlist-read-collaborative playlist-read-private streaming user-follow-modify user-follow-read user-library-read user-modify-playback-state user-read-currently-playing user-read-playback-position user-read-playback-state user-top-read',
 'expires_at': 1606908093}

### Track Audio Features

In [12]:
def track_audio_feat(user, all_tracks=None, top_tracks=None, term=None):
    """Generate DataFrame from user music library.
    
    Args:
        'user': Spotify user ID
        'all_tracks': if True, gathers all the track IDs for a
        given user's library
        'top_tracks': if True, gathers the user's top track IDs
        'term': The term should be specified; it allows to identify the
        term over which data is being acquired.  short_term refers to
        7 days; medium_term refers to 6 months; long_term is all time
    Returns:
        DataFrame containing the user's music tastes
    """
    artist_lst = []
    offset = 0
    for _ in range(2):
        top_artists = spot_session.current_user_top_artists()
    
    return None

In [51]:
trak = spot_session.track(track_id='0NeJjNlprGfZpeX2LQuN6c')
artst = spot_session.artist(trak['album']['artists'][0]['id'])
artst['genres']

['modern blues', 'soul']

In [53]:
genres_col = []
for track_id in df['id']:
    track_info = spot_session.track(track_id)
    artist = spot_session.artist(track_info['artists'][0]['id'])
    genres_col.append(artist['genres'])
len(genres_col)

KeyboardInterrupt: 

18286